what works?

-> it shows some graphic, and its kinda cool

To do list from here:

-> find more equation that might be necessary for the calculation, from the 5G_coverage_sim git repo

-> find out whats parameter on the dataset were for, how can we use them 

-> make sure that the calculation in this simulation were legit

-> make the program (based on this simulation), to run using parameters from the dataset

-> understanding the calculation in the code (to be put in the PDR)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display, clear_output
from filterpy.kalman import KalmanFilter

# Enable interactive mode for Jupyter
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 10)

class UAVTrackingSystem:
    def __init__(self, num_gnb=5, num_uav=3, area_size=(1000, 1000, 300), speed_of_light=3e8):
        self.num_gnb = num_gnb
        self.num_uav = num_uav
        self.area_size = area_size
        self.speed_of_light = speed_of_light
        self.carrier = {'Frequency': 100e6}
        self.prs = {'Signal': True}
        
        # Initialize positions
        self.gnb_positions, self.uav_positions = self.get_gnb_and_uav_positions()
        
        # Initialize Kalman filters for each UAV
        self.kalman_filters = [self.create_kalman_filter() for _ in range(num_uav)]
        
        # Storage for tracking data
        self.true_positions = []
        self.raw_positions = []
        self.filtered_positions = []
        self.errors_raw = []
        self.errors_filtered = []

    def create_kalman_filter(self):
        """Initialize a Kalman filter for 3D tracking"""
        kf = KalmanFilter(dim_x=6, dim_z=3)  # State: [x, y, z, vx, vy, vz], Measurement: [x, y, z]
        
        # State transition matrix (constant velocity model)
        dt = 1.0  # Time step
        kf.F = np.array([
            [1, 0, 0, dt, 0, 0],
            [0, 1, 0, 0, dt, 0],
            [0, 0, 1, 0, 0, dt],
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1]
        ])
        
        # Measurement matrix
        kf.H = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0]
        ])
        
        # Initial state covariance
        kf.P = np.diag([100, 100, 100, 10, 10, 10])
        
        # Measurement noise covariance
        kf.R = np.diag([10, 10, 10])
        
        # Process noise covariance
        kf.Q = np.diag([1, 1, 1, 0.5, 0.5, 0.5])
        
        # Initial state (center of area)
        kf.x = np.array([self.area_size[0]/2, self.area_size[1]/2, self.area_size[2]/2, 0, 0, 0])
        
        return kf

    def get_gnb_and_uav_positions(self):
        """Generate gNB and UAV positions"""
        gnb_positions = np.random.rand(self.num_gnb, 3) * self.area_size
        uav_positions = np.random.rand(self.num_uav, 3) * self.area_size
        return gnb_positions, uav_positions

    def generate_tx_waveform(self):
        """Generate transmission waveform"""
        time = np.linspace(0, 1, 1000)
        waveform = [np.sin(2 * np.pi * self.carrier['Frequency'] * time) for _ in range(self.num_gnb)]
        prs_grid = np.array(waveform)
        return np.array(waveform), prs_grid

    def compute_path_loss(self):
        """Compute path loss between gNBs and UAVs"""
        path_loss = np.zeros((self.num_gnb, self.num_uav))
        for i, gnb in enumerate(self.gnb_positions):
            for j, uav in enumerate(self.uav_positions):
                distance = np.linalg.norm(gnb - uav)
                path_loss[i, j] = 20 * np.log10(distance + 1e-6)
        return path_loss

    def estimate_toa_values(self):
        """Estimate Time of Arrival values"""
        toa_values = np.zeros((self.num_uav, self.num_gnb))
        for i, uav in enumerate(self.uav_positions):
            for j, gnb in enumerate(self.gnb_positions):
                distance = np.linalg.norm(gnb - uav)
                toa_values[i, j] = distance / self.speed_of_light
        return toa_values

    def assemble_toa_detection(self, toa_info, time):
        """Assemble TOA detections"""
        return [{"time": time, "toa": toa, "gnb": gnb} 
                for toa, gnb in zip(toa_info, self.gnb_positions)]

    def convert_toa_to_tdoa_detections(self, toa_detections):
        """Convert TOA to TDOA detections"""
        ref_toa = toa_detections[0]["toa"]
        return [{"tdoa": det["toa"] - ref_toa, "gnb": det["gnb"]} for det in toa_detections]

    def convert_tdoa_to_pos(self, tdoa_detections, true_position):
        """Convert TDOA to position with noise"""
        # This is a simplified version - in reality you'd use multilateration
        noise = np.random.normal(0, 10, size=3)  # Add some noise
        return true_position + noise

    def update_kalman_filter(self, kf, measurement):
        """Update Kalman filter with new measurement"""
        kf.predict()
        kf.update(measurement)
        return kf.x[:3]  # Return only position

    def generate_trajectory(self, num_points=100, traj_type='circle'):
        """Generate UAV trajectory"""
        if traj_type == 'circle':
            t = np.linspace(0, 2*np.pi, num_points)
            x = self.area_size[0]/2 + self.area_size[0]/3 * np.cos(t)
            y = self.area_size[1]/2 + self.area_size[1]/3 * np.sin(t)
            z = np.linspace(50, 250, num_points)
            return np.column_stack((x, y, z))
        elif traj_type == 'line':
            x = np.linspace(100, 900, num_points)
            y = np.linspace(100, 900, num_points)
            z = np.linspace(50, 250, num_points)
            return np.column_stack((x, y, z))
        else:  # random
            points = np.random.rand(num_points, 3) * self.area_size
            # Smooth the random path
            from scipy.ndimage import gaussian_filter1d
            for i in range(3):
                points[:, i] = gaussian_filter1d(points[:, i], sigma=2)
            return points

    def run_simulation(self, num_frames=100, traj_type='circle', noise_level=10):
        """Run complete simulation"""
        # Generate trajectory
        trajectory = self.generate_trajectory(num_frames, traj_type)
        
        # Reset storage
        self.true_positions = []
        self.raw_positions = []
        self.filtered_positions = []
        self.errors_raw = []
        self.errors_filtered = []
        
        # Reset Kalman filters
        for kf in self.kalman_filters:
            kf.x = np.array([self.area_size[0]/2, self.area_size[1]/2, self.area_size[2]/2, 0, 0, 0])
            kf.P = np.diag([100, 100, 100, 10, 10, 10])
        
        for i in range(num_frames):
            # Update UAV positions according to trajectory
            self.uav_positions = trajectory[i].reshape(1, 3)  # For single UAV
            
            # Simulate measurement process
            toa_info = self.estimate_toa_values()
            toa_detections = self.assemble_toa_detection(toa_info[0], i)  # First UAV only
            tdoa_detections = self.convert_toa_to_tdoa_detections(toa_detections)
            
            # Get position estimate (with noise)
            raw_position = self.convert_tdoa_to_pos(tdoa_detections, self.uav_positions[0])
            
            # Update Kalman filter
            filtered_position = self.update_kalman_filter(self.kalman_filters[0], raw_position)
            
            # Store results
            self.true_positions.append(self.uav_positions[0])
            self.raw_positions.append(raw_position)
            self.filtered_positions.append(filtered_position)
            
            # Calculate errors
            self.errors_raw.append(np.linalg.norm(raw_position - self.uav_positions[0]))
            self.errors_filtered.append(np.linalg.norm(filtered_position - self.uav_positions[0]))
        
        # Convert to numpy arrays
        self.true_positions = np.array(self.true_positions)
        self.raw_positions = np.array(self.raw_positions)
        self.filtered_positions = np.array(self.filtered_positions)
        
        # Calculate RMSE
        rmse_raw = np.sqrt(np.mean(self.errors_raw)**2)
        rmse_filtered = np.sqrt(np.mean(self.errors_filtered)**2)
        
        return rmse_raw, rmse_filtered

def run_interactive_simulation():
    """Run interactive simulation with widgets"""
    # Create widgets
    num_gnb_slider = widgets.IntSlider(
        value=5, min=3, max=10, step=1, description='gNBs:'
    )
    
    num_frames_slider = widgets.IntSlider(
        value=50, min=10, max=200, step=10, description='Frames:'
    )
    
    traj_dropdown = widgets.Dropdown(
        options=['circle', 'line', 'random'],
        value='circle',
        description='Trajectory:'
    )
    
    noise_slider = widgets.FloatSlider(
        value=10, min=1, max=50, step=1, description='Noise:'
    )
    
    frame_slider = widgets.IntSlider(
        value=0, min=0, max=num_frames_slider.value-1, step=1, description='Frame:'
    )
    
    play_button = widgets.Play(
        value=0, min=0, max=num_frames_slider.value-1, step=1, interval=100, description="Play"
    )
    
    run_button = widgets.Button(
        description='Run Simulation', button_style='success'
    )
    
    # Link widgets
    widgets.jslink((play_button, 'value'), (frame_slider, 'value'))
    
    def update_frame_max(*args):
        frame_slider.max = num_frames_slider.value-1
        play_button.max = num_frames_slider.value-1
    
    num_frames_slider.observe(update_frame_max, 'value')
    
    # Create output area
    output = widgets.Output()
    
    # Create tracking system
    tracker = UAVTrackingSystem()
    
    def on_run_button_clicked(b):
        with output:
            output.clear_output(wait=True)
            
            # Update tracker parameters
            tracker.num_gnb = num_gnb_slider.value
            tracker.gnb_positions, _ = tracker.get_gnb_and_uav_positions()
            
            # Run simulation
            rmse_raw, rmse_filtered = tracker.run_simulation(
                num_frames=num_frames_slider.value,
                traj_type=traj_dropdown.value,
                noise_level=noise_slider.value
            )
            
            print(f"Simulation completed with {num_frames_slider.value} frames")
            print(f"RMSE (Raw): {rmse_raw:.2f} m")
            print(f"RMSE (Filtered): {rmse_filtered:.2f} m")
            print(f"Improvement: {(1 - rmse_filtered/rmse_raw)*100:.1f}%")
            
            # Update plot
            update_plot(0)
    
    def update_plot(frame):
        with output:
            output.clear_output(wait=True)
            
            if len(tracker.true_positions) == 0:
                print("Please run the simulation first.")
                return
            
            fig = plt.figure(figsize=(16, 6))
            
            # 3D Tracking Plot
            ax1 = fig.add_subplot(121, projection='3d')
            
            # Plot gNBs
            ax1.scatter(
                tracker.gnb_positions[:, 0], 
                tracker.gnb_positions[:, 1], 
                tracker.gnb_positions[:, 2], 
                c='r', marker='o', s=100, label='gNB'
            )
            
            # Plot true trajectory
            ax1.plot(
                tracker.true_positions[:frame+1, 0],
                tracker.true_positions[:frame+1, 1],
                tracker.true_positions[:frame+1, 2],
                'g-', label='True Trajectory'
            )
            
            # Plot raw estimates
            ax1.plot(
                tracker.raw_positions[:frame+1, 0],
                tracker.raw_positions[:frame+1, 1],
                tracker.raw_positions[:frame+1, 2],
                'r:', label='Raw Estimates'
            )
            
            # Plot filtered estimates
            ax1.plot(
                tracker.filtered_positions[:frame+1, 0],
                tracker.filtered_positions[:frame+1, 1],
                tracker.filtered_positions[:frame+1, 2],
                'b-', label='Kalman Filter'
            )
            
            # Plot current positions
            if frame < len(tracker.true_positions):
                ax1.scatter(
                    [tracker.true_positions[frame, 0]],
                    [tracker.true_positions[frame, 1]],
                    [tracker.true_positions[frame, 2]],
                    c='g', s=150, marker='*', label='True Position'
                )
                
                ax1.scatter(
                    [tracker.raw_positions[frame, 0]],
                    [tracker.raw_positions[frame, 1]],
                    [tracker.raw_positions[frame, 2]],
                    c='r', s=100, marker='o', label='Raw Estimate'
                )
                
                ax1.scatter(
                    [tracker.filtered_positions[frame, 0]],
                    [tracker.filtered_positions[frame, 1]],
                    [tracker.filtered_positions[frame, 2]],
                    c='b', s=100, marker='o', label='Filtered Estimate'
                )
            
            ax1.set_xlim(0, tracker.area_size[0])
            ax1.set_ylim(0, tracker.area_size[1])
            ax1.set_zlim(0, tracker.area_size[2])
            ax1.set_xlabel('X (m)')
            ax1.set_ylabel('Y (m)')
            ax1.set_zlabel('Z (m)')
            ax1.set_title(f'3D UAV Tracking (Frame {frame+1}/{num_frames_slider.value})')
            ax1.legend()
            
            # Error Plot
            ax2 = fig.add_subplot(122)
            
            ax2.plot(
                np.arange(len(tracker.errors_raw[:frame+1])),
                tracker.errors_raw[:frame+1],
                'r-', label='Raw Error'
            )
            
            ax2.plot(
                np.arange(len(tracker.errors_filtered[:frame+1])),
                tracker.errors_filtered[:frame+1],
                'b-', label='Filtered Error'
            )
            
            # Calculate and plot moving averages
            window = min(10, len(tracker.errors_raw[:frame+1]))
            if window > 0:
                raw_moving_avg = np.convolve(tracker.errors_raw[:frame+1], np.ones(window)/window, mode='valid')
                filtered_moving_avg = np.convolve(tracker.errors_filtered[:frame+1], np.ones(window)/window, mode='valid')
                
                x_vals = np.arange(window-1, len(tracker.errors_raw[:frame+1]))
                ax2.plot(x_vals, raw_moving_avg, 'r--', label=f'Raw MA ({window})')
                ax2.plot(x_vals, filtered_moving_avg, 'b--', label=f'Filtered MA ({window})')
            
            ax2.set_xlabel('Time Step')
            ax2.set_ylabel('Position Error (m)')
            ax2.set_title('Tracking Error Over Time')
            ax2.grid(True)
            ax2.legend()
            
            plt.tight_layout()
            plt.show()
    
    def on_frame_change(change):
        update_plot(change.new)
    
    frame_slider.observe(on_frame_change, names='value')
    run_button.on_click(on_run_button_clicked)
    
    # Arrange widgets
    controls = widgets.VBox([
        widgets.HBox([num_gnb_slider, num_frames_slider]),
        widgets.HBox([traj_dropdown, noise_slider]),
        widgets.HBox([play_button, frame_slider]),
        run_button
    ])
    
    display(controls)
    display(output)
    
    with output:
        print("Click 'Run Simulation' to begin.")

# Run the interactive simulation
run_interactive_simulation()

Output()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.linalg import norm
from filterpy.kalman import KalmanFilter
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from matplotlib.animation import FuncAnimation
import matplotlib

# Enable interactive mode for Jupyter
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 10)

# Define constants (made adjustable via widgets)
DEFAULT_SPEED_OF_SOUND = 3e8  # m/s (speed of light - default for RF signals)
DEFAULT_NOISE_STDDEV = 0.5    # Standard deviation of noise (for simulation)

class TDOATracker:
    def __init__(self, receiver_positions, signal_speed=DEFAULT_SPEED_OF_SOUND, noise_stddev=DEFAULT_NOISE_STDDEV):
        """
        Initialize the tracker with receiver positions.
        """
        self.receivers = np.array(receiver_positions)
        self.num_receivers = len(self.receivers)
        self.signal_speed = signal_speed
        self.noise_stddev = noise_stddev
        
    def calculate_true_tdoa(self, object_position):
        """
        Calculate the true TDOA values based on the distance between the object and the receivers.
        """
        distances = np.linalg.norm(self.receivers - object_position, axis=1)
        tdoa = (distances - distances[0]) / self.signal_speed
        return tdoa[1:]

    def add_noise_to_tdoa(self, true_tdoa):
        """
        Add Gaussian noise to the TDOA values to simulate real-world conditions.
        """
        noise = np.random.normal(0, self.noise_stddev, size=true_tdoa.shape)
        noisy_tdoa = true_tdoa + noise
        return noisy_tdoa
    
    def tdoa_position_estimation(self, noisy_tdoa, max_iters=100):
        """
        Estimate the position of the object using least squares optimization from noisy TDOA measurements.
        """
        def residuals(x):
            predicted_tdoa = self.calculate_true_tdoa(x)
            return predicted_tdoa - noisy_tdoa
        
        initial_guess = np.mean(self.receivers, axis=0)
        
        try:
            result = least_squares(residuals, initial_guess, max_nfev=max_iters,
                                  method='trf', loss='soft_l1')
            return result.x
        except:
            return initial_guess

class VehicleTrackingSystem:
    def __init__(self, receiver_positions, signal_speed=DEFAULT_SPEED_OF_SOUND, 
                 noise_stddev=DEFAULT_NOISE_STDDEV, process_noise=0.1):
        """
        Initialize the vehicle tracking system.
        """
        self.tracker = TDOATracker(receiver_positions, signal_speed, noise_stddev)
        
        # Initialize Kalman filter
        self.kf = KalmanFilter(dim_x=4, dim_z=2)  # State: [x, y, vx, vy], Measurement: [x, y]
        
        # State transition matrix (constant velocity model)
        self.kf.F = np.array([[1, 0, 1, 0],
                              [0, 1, 0, 1],
                              [0, 0, 1, 0],
                              [0, 0, 0, 1]])
        
        # Measurement matrix
        self.kf.H = np.array([[1, 0, 0, 0],
                              [0, 1, 0, 0]])
        
        # Initial state covariance
        self.kf.P = np.diag([100, 100, 10, 10])

        # Measurement noise covariance
        self.kf.R = np.eye(2) * (noise_stddev ** 2)

        # Process noise covariance
        self.kf.Q = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 10, 0],
            [0, 0, 0, 10]
        ]) * process_noise
        
        # Initialize state
        mean_pos = np.mean(receiver_positions, axis=0)
        self.kf.x = np.array([mean_pos[0], mean_pos[1], 0, 0])
        
        # Store results
        self.raw_measurements = []
        self.filtered_positions = []
        self.errors_raw = []
        self.errors_filtered = []

    def reset(self, noise_stddev=None, process_noise=None):
        """Reset the Kalman filter and clear history."""
        if noise_stddev is not None:
            self.tracker.noise_stddev = noise_stddev
            self.kf.R = np.eye(2) * (noise_stddev ** 2)
            
        if process_noise is not None:
            self.kf.Q = np.array([
                [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 10, 0],
                [0, 0, 0, 10]
            ]) * process_noise
            
        # Reset Kalman filter state
        mean_pos = np.mean(self.tracker.receivers, axis=0)
        self.kf.x = np.array([mean_pos[0], mean_pos[1], 0, 0])
        self.kf.P = np.diag([100, 100, 10, 10])
        
        # Clear history
        self.raw_measurements = []
        self.filtered_positions = []
        self.errors_raw = []
        self.errors_filtered = []

    def estimate_position(self, true_position):
        """
        Estimate the position using TDOA measurements from the true position.
        """
        true_tdoa = self.tracker.calculate_true_tdoa(true_position)
        noisy_tdoa = self.tracker.add_noise_to_tdoa(true_tdoa)
        estimated_position = self.tracker.tdoa_position_estimation(noisy_tdoa)
        
        self.raw_measurements.append(estimated_position)
        error_raw = np.linalg.norm(estimated_position - true_position)
        self.errors_raw.append(error_raw)
        
        return estimated_position

    def update_kalman_filter(self, estimated_position, true_position):
        """
        Update Kalman filter with a new measurement.
        """
        self.kf.predict()
        
        # Simplified measurement update
        self.kf.update(estimated_position)
        
        filtered_position = self.kf.x[:2]
        self.filtered_positions.append(filtered_position)
        
        error_filtered = np.linalg.norm(filtered_position - true_position)
        self.errors_filtered.append(error_filtered)
        
        return filtered_position

    def process_measurement(self, true_position):
        """
        Process a new measurement through the complete pipeline.
        """
        raw_position = self.estimate_position(true_position)
        filtered_position = self.update_kalman_filter(raw_position, true_position)
        return raw_position, filtered_position

# (Keep your existing generate_receiver_layout and generate_vehicle_trajectory functions)

def run_interactive_simulation(area_size=100, max_frames=100):
    """
    Run an interactive simulation with widgets for parameter control.
    """
    # Create widgets
    layout_dropdown = widgets.Dropdown(
        options=['square', 'triangle', 'pentagon', 'random'],
        value='square',
        description='Receivers:',
    )
    
    traj_dropdown = widgets.Dropdown(
        options=['line', 'circle', 'figure8', 'square', 'random'],
        value='circle',
        description='Trajectory:',
    )
    
    noise_slider = widgets.FloatSlider(
        value=0.5,
        min=0.1,
        max=5.0,
        step=0.1,
        description='Noise:',
    )
    
    process_noise_slider = widgets.FloatSlider(
        value=0.1,
        min=0.01,
        max=1.0,
        step=0.01,
        description='Proc. Noise:',
    )
    
    frames_slider = widgets.IntSlider(
        value=50,
        min=10,
        max=max_frames,
        step=5,
        description='Frames:',
    )
    
    frame_selector = widgets.IntSlider(
        value=0,
        min=0,
        max=frames_slider.value-1,
        step=1,
        description='Frame:',
    )
    
    play_button = widgets.Play(
        value=0,
        min=0,
        max=frames_slider.value-1,
        step=1,
        interval=100,
        description="Play",
    )
    
    show_errors_checkbox = widgets.Checkbox(
        value=True,
        description='Show Errors',
    )
    
    run_button = widgets.Button(
        description='Run Simulation',
        button_style='success'
    )
    
    # Link widgets
    widgets.jslink((play_button, 'value'), (frame_selector, 'value'))
    
    def update_frame_max(*args):
        frame_selector.max = frames_slider.value-1
        play_button.max = frames_slider.value-1
    
    frames_slider.observe(update_frame_max, 'value')
    
    # Output area
    output = widgets.Output()
    
    # Simulation data
    simulation_data = {
        'receivers': None,
        'trajectory': None,
        'tracker': None,
        'raw_positions': [],
        'filtered_positions': []
    }
    
    def on_run_button_clicked(b):
        with output:
            output.clear_output(wait=True)
            
            num_frames = frames_slider.value
            
            # Generate receiver layout
            simulation_data['receivers'] = generate_receiver_layout(
                layout_dropdown.value, area_size, variation=2
            )
            
            # Generate vehicle trajectory
            simulation_data['trajectory'] = generate_vehicle_trajectory(
                traj_dropdown.value, num_frames, area_size
            )
            
            # Create tracking system
            simulation_data['tracker'] = VehicleTrackingSystem(
                simulation_data['receivers'],
                DEFAULT_SPEED_OF_SOUND,
                noise_slider.value,
                process_noise_slider.value
            )
            
            # Process each trajectory point
            simulation_data['raw_positions'] = []
            simulation_data['filtered_positions'] = []
            
            for point in simulation_data['trajectory']:
                raw_pos, filtered_pos = simulation_data['tracker'].process_measurement(point)
                simulation_data['raw_positions'].append(raw_pos)
                simulation_data['filtered_positions'].append(filtered_pos)
            
            # Convert to numpy arrays
            simulation_data['raw_positions'] = np.array(simulation_data['raw_positions'])
            simulation_data['filtered_positions'] = np.array(simulation_data['filtered_positions'])
            
            # Calculate errors
            rmse_raw = np.sqrt(np.mean(np.sum((simulation_data['trajectory'] - simulation_data['raw_positions']) ** 2, axis=1)))
            rmse_filtered = np.sqrt(np.mean(np.sum((simulation_data['trajectory'] - simulation_data['filtered_positions']) ** 2, axis=1)))
            
            print(f"Simulation with {num_frames} frames")
            print(f"RMSE (Raw): {rmse_raw:.4f} m")
            print(f"RMSE (Filtered): {rmse_filtered:.4f} m")
            print(f"Improvement: {(1 - rmse_filtered/rmse_raw) * 100:.2f}%")
            
            # Reset frame selector
            frame_selector.value = 0
            update_plot(0)
    
    def update_plot(frame):
        with output:
            output.clear_output(wait=True)
            
            if simulation_data['trajectory'] is None:
                print("Please run the simulation first.")
                return
            
            fig = plt.figure(figsize=(14, 8))
            ax1 = fig.add_subplot(1, 2, 1)
            
            # Set reasonable axis limits
            all_points = np.vstack([
                simulation_data['trajectory'],
                simulation_data['raw_positions'],
                simulation_data['filtered_positions']
            ])
            x_min, y_min = np.min(all_points, axis=0) - 5
            x_max, y_max = np.max(all_points, axis=0) + 5
            ax1.set_xlim(max(0, x_min), min(area_size, x_max))
            ax1.set_ylim(max(0, y_min), min(area_size, y_max))
            
            # Plot trajectory and estimates
            ax1.plot(simulation_data['trajectory'][:, 0], simulation_data['trajectory'][:, 1], 
                    'g-', alpha=0.7, label="True Trajectory")
            ax1.plot(simulation_data['raw_positions'][:frame+1, 0], 
                    simulation_data['raw_positions'][:frame+1, 1], 
                    'r:', alpha=0.8, label="Raw Estimates")
            ax1.plot(simulation_data['filtered_positions'][:frame+1, 0], 
                    simulation_data['filtered_positions'][:frame+1, 1], 
                    'b-', linewidth=2, label="Kalman Filter")
            
            # Plot receivers
            receivers = simulation_data['receivers']
            ax1.scatter(receivers[:, 0], receivers[:, 1], c='black', marker='s', s=80, label="Receivers")
            for i, (x, y) in enumerate(receivers):
                ax1.annotate(f"{i+1}", (x, y), fontsize=12, ha='center', va='center', color='white')
            
            # Plot current positions
            if frame < len(simulation_data['trajectory']):
                true_pos = simulation_data['trajectory'][frame]
                raw_pos = simulation_data['raw_positions'][frame]
                filtered_pos = simulation_data['filtered_positions'][frame]
                
                ax1.scatter([true_pos[0]], [true_pos[1]], c='green', s=100, marker='o',
                           edgecolor='black', label="True Position")
                ax1.scatter([raw_pos[0]], [raw_pos[1]], c='red', s=80, marker='o',
                           edgecolor='black', label="Raw Estimate")
                ax1.scatter([filtered_pos[0]], [filtered_pos[1]], c='blue', s=80, marker='o',
                           edgecolor='black', label="Filtered Estimate")
            
            ax1.set_title(f"2D TDOA Position Tracking (Frame {frame+1}/{frames_slider.value})")
            ax1.set_xlabel("X Position (m)")
            ax1.set_ylabel("Y Position (m)")
            ax1.grid(True)
            ax1.legend(loc='upper right')
            
            # Error plot
            if show_errors_checkbox.value:
                ax2 = fig.add_subplot(1, 2, 2)
                errors_raw = simulation_data['tracker'].errors_raw[:frame+1]
                errors_filtered = simulation_data['tracker'].errors_filtered[:frame+1]
                
                ax2.plot(np.arange(len(errors_raw)), errors_raw, 'r-', alpha=0.7, label="Raw Error")
                ax2.plot(np.arange(len(errors_filtered)), errors_filtered, 'b-', linewidth=2, label="Filtered Error")
                
                if len(errors_raw) > 0:
                    window = min(10, len(errors_raw))
                    if window > 0:
                        raw_moving_avg = np.convolve(errors_raw, np.ones(window)/window, mode='valid')
                        filtered_moving_avg = np.convolve(errors_filtered, np.ones(window)/window, mode='valid')
                        
                        x_vals = np.arange(window-1, len(errors_raw))
                        ax2.plot(x_vals, raw_moving_avg, 'r--', linewidth=2, label=f"Raw Avg ({window} points)")
                        ax2.plot(x_vals, filtered_moving_avg, 'b--', linewidth=2, 
                                label=f"Filtered Avg ({window} points)")
                
                ax2.set_title("Position Error over Time")
                ax2.set_xlabel("Time Step")
                ax2.set_ylabel("Error (m)")
                ax2.grid(True)
                ax2.legend(loc='upper right')
                
                if len(errors_raw) > 0:
                    max_error = max(max(errors_raw), max(errors_filtered)) * 1.2
                    ax2.set_ylim(0, max_error)
            
            plt.tight_layout()
            plt.show()
    
    def on_frame_change(change):
        update_plot(change.new)
    
    frame_selector.observe(on_frame_change, names='value')
    run_button.on_click(on_run_button_clicked)
    
    # Arrange widgets
    controls = widgets.VBox([
        widgets.HBox([layout_dropdown, traj_dropdown]),
        widgets.HBox([noise_slider, process_noise_slider]),
        frames_slider,
        widgets.HBox([play_button, frame_selector]),
        show_errors_checkbox,
        run_button
    ])
    
    display(controls)
    display(output)
    
    with output:
        print("Click 'Run Simulation' to begin.")

# Run the simulation with up to 200 frames possible
run_interactive_simulation(area_size=100, max_frames=200)

Output()